In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array
from zarr.storage import FSStore
import fsspec
from fsspec.implementations.local import LocalFileSystem

In [3]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="sauchdev",
    anon=False,
)

In [ ]:
fs.download("az://data/zarr_templates/latest_nwp_sample.nc", "latest_nwp_sample_001.nc")

sample = xr.open_dataset("latest_nwp_sample_001.nc")

In [3]:
template = xr.Dataset(None, coords=sample.coords)

In [4]:
for name in template._coord_names:
    if np.issubdtype(template[name].dtype, np.datetime64):
        template[name].encoding["units"] = "hours since 2023-01-01 00:00:00"
    if np.issubdtype(template[name].dtype, np.timedelta64):
        template[name].encoding["units"] = "hours"

In [5]:
time_utc_idx = pd.date_range(start="2023-01-01T00:00:00", end="2026-01-01T00:00:00", freq=pd.Timedelta(hours=1))

In [6]:
template = template.reindex(time_utc=time_utc_idx)

In [7]:
shape = tuple(v for v in template.coords.sizes.values())

In [8]:
dims = tuple(k for k in template.dims.keys())

/tmp/ipykernel_32591/3042545308.py:1: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dims = tuple(k for k in template.dims.keys())


## Temp store

In [9]:
empty_array = dask.array.empty(shape=shape, chunks=[shape[0], 24, 50, 50])

In [10]:
for variable in sample.data_vars:
    template[variable] = (dims, empty_array)

In [ ]:
remote_store = FSStore(
    url="az://data/tmp_latest_nwp_rechunked",
    fs=fs,
)

In [13]:
template.to_zarr(
    store=remote_store,
    compute=False,
)

Delayed('_finalize_store-6cb429a3-3c55-490a-a6c0-c6849dcf1cb0')

## Final store

In [9]:
empty_array = dask.array.empty(shape=shape, chunks=[shape[0], shape[1], 50, 50])

In [10]:
for variable in sample.data_vars:
    template[variable] = (dims, empty_array)

In [15]:
# need to add x and y as coordinates, in order to write in regions for these blocks
template = template.assign_coords(
    x=template["x"],
    y=template["y"],
)

In [ ]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="sauchdev",
    anon=False,
)

remote_store = FSStore(
    url="az://data/latest_nwp_rechunked",
    fs=fs,
)

In [18]:
template.to_zarr(
    store=remote_store,
    compute=False,
)

Delayed('_finalize_store-420741b5-ba26-44cb-8b87-309383a97487')